## This notebook demonstratres the Data Transformation procedure

In [1]:
import pandas as pd
import warnings
import os
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
import joblib

In [2]:
parent_dir = os.path.dirname(os.getcwd())
data_path = os.path.join(parent_dir, 'data', 'datasets')
out_path = os.path.join(parent_dir, 'data', 'transform')

Load the cleaned matchups dataset, the champion role metrics, and the champion metrics datasets

In [3]:
dataset = pd.read_csv(os.path.join(data_path, 'dataset.csv'))
champ_role_metrics = pd.read_csv(os.path.join(data_path, 'champ_role_metrics.csv'))
champ_metrics = pd.read_csv(os.path.join(data_path, 'champ_metrics.csv'))

### Filter matches with suboptimal data
Through preprocessing and generating the metrics datasets we found that not all combinations of champion and roles have enough data to generate metrics.
Create a champ_role column to filter out champion role combinations we don't have metrics for

In [4]:
dataset_filtered = dataset.copy()
dataset_filtered['champ_role'] = dataset_filtered['CHAMPION'].str.cat(dataset_filtered[['ROLE']], sep='_')

Get all champion role combinations we have metrics for. 

156 x 5 = 780 different champion role combinations are possible.

However, valid data is only available for 565 different combinations. 

In [5]:
valid_champ_role = champ_role_metrics['CHAMPION'].str.cat(champ_role_metrics[['ROLE']], sep='_').unique().tolist()
len(valid_champ_role)

565

Drop all records of champion role combinations for which metrics are unavailable

In [6]:
dataset_filtered = dataset_filtered[dataset_filtered.champ_role.isin(valid_champ_role)]

Each match_id must have 10 records associated with it. One for each player in that match. 

However, not all matches have all 10 records. Only records from those matches that have 10 records will be kept.

In [7]:
match_counts = dataset_filtered.MATCH_ID.value_counts().reset_index(name='count')
matches_less_than_10 = match_counts[match_counts['count'] != 10]['index'].tolist()
len(matches_less_than_10)

7241

Only champion name, role, win, and match_id columns are required from the matchups dataset for the final transformation. 

In [8]:
dataset_filtered = dataset_filtered[~dataset_filtered.MATCH_ID.isin(matches_less_than_10)]
dataset_filtered = dataset_filtered[['CHAMPION', 'MATCH_ID', 'ROLE', 'champ_role', 'WIN']]
dataset_filtered.shape

(1262730, 5)

Get a list of all the unique match_ids in the dataset. Each match_id will have one record associated with it in the final dataset.

In [9]:
unique_matches = dataset_filtered.MATCH_ID.unique().tolist()
len(unique_matches)

126273

### Transform Data
The transformed dataset will have the columns defined below in list 'features'

t1 and t2 refer to the two differnt teams in a match

The role winrate and role kda for each champion will be retrieved from the champion role metrics dataset

The winrate and kda for each champion will be retrieved from the champion metrics dataset

t1_win is the prediction lable, this describes whether team1 won or lost.

In [10]:
features = ['t1_top','t1_middle','t1_bottom','t1_utility','t1_jungle',
    't2_top','t2_middle','t2_bottom','t2_utility','t2_jungle',
    
    't1_top_wr','t1_middle_wr','t1_bottom_wr','t1_utility_wr','t1_jungle_wr',
    't2_top_wr','t2_middle_wr','t2_bottom_wr','t2_utility_wr','t2_jungle_wr',

    't1_top_role_wr','t1_middle_role_wr','t1_bottom_role_wr','t1_utility_role_wr','t1_jungle_role_wr',
    't2_top_role_wr','t2_middle_role_wr','t2_bottom_role_wr','t2_utility_role_wr','t2_jungle_role_wr',

    't1_top_kda','t1_middle_kda','t1_bottom_kda','t1_utility_kda','t1_jungle_kda',
    't2_top_kda','t2_middle_kda','t2_bottom_kda','t2_utility_kda','t2_jungle_kda',

    't1_top_role_kda','t1_middle_role_kda','t1_bottom_role_kda','t1_utility_role_kda','t1_jungle_role_kda',
    't2_top_role_kda','t2_middle_role_kda','t2_bottom_role_kda','t2_utility_role_kda','t2_jungle_role_kda',            
    
    't1_win']

Perform the transformation

In [12]:
tf_df = pd.DataFrame(columns = features)
for i in range(4):       
    record = {}
    temp = dataset_filtered[dataset_filtered.MATCH_ID == unique_matches[i]]

    if i%2 == 0:
        for index, row in temp.iterrows():
            if row['WIN'] == True:
                record["t1_"+row['ROLE']] = row['CHAMPION']

                record["t1_"+row['ROLE']+"_wr"] = champ_metrics[champ_metrics['CHAMPION'] == row['CHAMPION']]['wr'].iloc[0]
                record["t1_"+row['ROLE']+"_kda"] = champ_metrics[champ_metrics['CHAMPION'] == row['CHAMPION']]['kda'].iloc[0]

                record["t1_"+row['ROLE']+"_role_wr"] = champ_role_metrics[(champ_role_metrics['CHAMPION'] == row['CHAMPION']) & (champ_role_metrics['ROLE'] == row['ROLE'])]['wr'].iloc[0]
                record["t1_"+row['ROLE']+"_role_kda"] =  champ_role_metrics[(champ_role_metrics['CHAMPION'] == row['CHAMPION']) & (champ_role_metrics['ROLE'] == row['ROLE'])]['kda'].iloc[0]

            elif row['WIN'] == False:
                record["t2_"+row['ROLE']] = row['CHAMPION']

                record["t2_"+row['ROLE']+"_wr"] = champ_metrics[champ_metrics['CHAMPION'] == row['CHAMPION']]['wr'].iloc[0]
                record["t2_"+row['ROLE']+"_kda"] = champ_metrics[champ_metrics['CHAMPION'] == row['CHAMPION']]['kda'].iloc[0]

                record["t2_"+row['ROLE']+"_role_wr"] = champ_role_metrics[(champ_role_metrics['CHAMPION'] == row['CHAMPION']) & (champ_role_metrics['ROLE'] == row['ROLE'])]['wr'].iloc[0]
                record["t2_"+row['ROLE']+"_role_kda"] =  champ_role_metrics[(champ_role_metrics['CHAMPION'] == row['CHAMPION']) & (champ_role_metrics['ROLE'] == row['ROLE'])]['kda'].iloc[0]
                
            
        record["t1_win"] = True
    else:
        for index, row in temp.iterrows():
            if row['WIN'] == False:
                record["t1_"+row['ROLE']] = row['CHAMPION']

                record["t1_"+row['ROLE']+"_wr"] = champ_metrics[champ_metrics['CHAMPION'] == row['CHAMPION']]['wr'].iloc[0]
                record["t1_"+row['ROLE']+"_kda"] = champ_metrics[champ_metrics['CHAMPION'] == row['CHAMPION']]['kda'].iloc[0]

                record["t1_"+row['ROLE']+"_role_wr"] = champ_role_metrics[(champ_role_metrics['CHAMPION'] == row['CHAMPION']) & (champ_role_metrics['ROLE'] == row['ROLE'])]['wr'].iloc[0]
                record["t1_"+row['ROLE']+"_role_kda"] =  champ_role_metrics[(champ_role_metrics['CHAMPION'] == row['CHAMPION']) & (champ_role_metrics['ROLE'] == row['ROLE'])]['kda'].iloc[0]

            elif row['WIN'] == True:
                record["t2_"+row['ROLE']] = row['CHAMPION']

                record["t2_"+row['ROLE']+"_wr"] = champ_metrics[champ_metrics['CHAMPION'] == row['CHAMPION']]['wr'].iloc[0]
                record["t2_"+row['ROLE']+"_kda"] = champ_metrics[champ_metrics['CHAMPION'] == row['CHAMPION']]['kda'].iloc[0]

                record["t2_"+row['ROLE']+"_role_wr"] = champ_role_metrics[(champ_role_metrics['CHAMPION'] == row['CHAMPION']) & (champ_role_metrics['ROLE'] == row['ROLE'])]['wr'].iloc[0]
                record["t2_"+row['ROLE']+"_role_kda"] =  champ_role_metrics[(champ_role_metrics['CHAMPION'] == row['CHAMPION']) & (champ_role_metrics['ROLE'] == row['ROLE'])]['kda'].iloc[0]

        record["t1_win"] = False      
    tf_df = tf_df.append(record, ignore_index=True)


4 records from the transformed dataset are shown below.

Each record corresponds to a single match and contains the champion being played, winrate of that champion, kda of that champion, winrate of the champion in the role it is being played, the k/da of the champion in the role it is being played, for each of the 10 players in the match

In [13]:
tf_df

,t1_top,t1_middle,t1_bottom,t1_utility,t1_jungle,t2_top,t2_middle,t2_bottom,t2_utility,t2_jungle,t1_top_wr,t1_middle_wr,t1_bottom_wr,t1_utility_wr,t1_jungle_wr,t2_top_wr,t2_middle_wr,t2_bottom_wr,t2_utility_wr,t2_jungle_wr,t1_top_role_wr,t1_middle_role_wr,t1_bottom_role_wr,t1_utility_role_wr,t1_jungle_role_wr,t2_top_role_wr,t2_middle_role_wr,t2_bottom_role_wr,t2_utility_role_wr,t2_jungle_role_wr,t1_top_kda,t1_middle_kda,t1_bottom_kda,t1_utility_kda,t1_jungle_kda,t2_top_kda,t2_middle_kda,t2_bottom_kda,t2_utility_kda,t2_jungle_kda,t1_top_role_kda,t1_middle_role_kda,t1_bottom_role_kda,t1_utility_role_kda,t1_jungle_role_kda,t2_top_role_kda,t2_middle_role_kda,t2_bottom_role_kda,t2_utility_role_kda,t2_jungle_role_kda,t1_win
0,Fiora,Corki,Samira,Pantheon,Kayn,Kassadin,Yone,Kaisa,Nautilus,Evelynn,52.233364,51.429580,49.760483,48.664122,50.689906,51.263264,48.790814,48.928255,49.915314,49.061958,52.334540,51.168224,49.843517,45.492289,50.805617,56.422018,48.712121,48.938667,50.100690,49.095238,1.827452,2.412114,2.293444,2.151817,2.611583,2.582077,1.901734,2.531856,2.354769,2.940007,1.827214,2.455387,2.301646,1.936946,2.627467,2.400353,1.981926,2.534069,2.355415,2.950438,True
1,Camille,Teemo,Tristana,Yuumi,Viego,Singed,Yone,Jhin,Pyke,Taliyah,52.311781,47.116516,51.078886,49.761747,49.082066,50.407886,48.790814,49.916269,49.529812,50.834221,52.248311,43.497758,51.250478,49.827638,49.100754,50.867721,48.712121,49.941680,49.052237,51.181453,2.095402,1.733938,2.467593,4.286643,2.414268,2.223274,1.901734,2.889686,2.268547,2.684889,2.088099,1.906325,2.510202,4.290919,2.595813,2.204869,1.981926,2.892598,2.214440,2.816094,False
2,Ornn,Zed,Jinx,Lulu,FiddleSticks,Shen,Sylas,Samira,Leona,XinZhao,49.782230,49.523810,50.973808,50.715869,51.464693,51.536406,50.200375,49.760483,50.387911,51.869731,49.851088,49.647516,50.998588,50.813931,51.669086,51.637952,50.421731,49.843517,50.450267,52.010744,2.439610,2.277548,2.504318,3.207114,2.796019,2.912215,2.127615,2.293444,2.442175,2.538234,2.436134,2.282057,2.505360,3.213491,2.843977,2.933312,2.200415,2.301646,2.443675,2.574962,True
3,Jayce,Sylas,Ezreal,Yuumi,Kayn,Volibear,AurelionSol,Caitlyn,Shaco,Viego,47.357867,50.200375,49.461616,49.761747,50.689906,48.582135,52.156177,48.906789,50.079104,49.082066,47.281568,50.421731,49.573524,49.827638,50.805617,48.416390,52.816020,48.984722,47.442958,49.100754,1.812713,2.127615,2.577849,4.286643,2.611583,2.351976,2.690024,2.326337,2.705159,2.414268,1.753307,2.200415,2.586941,4.290919,2.627467,1.927295,2.735680,2.331698,2.396533,2.595813,False


The first row of the transformed dataset corresponds to the following rows from the filtered matchups data 

In [14]:
dataset_filtered[dataset_filtered.MATCH_ID == unique_matches[0]]

,CHAMPION,MATCH_ID,ROLE,champ_role,WIN
0,Kassadin,1201957752_TR1,top,Kassadin_top,False
1,Fiora,1201957752_TR1,top,Fiora_top,True
2,Yone,1201957752_TR1,middle,Yone_middle,False
3,Corki,1201957752_TR1,middle,Corki_middle,True
4,Kaisa,1201957752_TR1,bottom,Kaisa_bottom,False
5,Samira,1201957752_TR1,bottom,Samira_bottom,True
6,Nautilus,1201957752_TR1,utility,Nautilus_utility,False
7,Pantheon,1201957752_TR1,utility,Pantheon_utility,True
8,Evelynn,1201957752_TR1,jungle,Evelynn_jungle,False
9,Kayn,1201957752_TR1,jungle,Kayn_jungle,True


Since applying this transform on such a large dataset can take several hours, the data was split into 5 parts and each part was transformed in a seperate notebook in parallel. The transformed parts were finally merged into one large dataset.